In [1]:
import numpy as np
import SearchSpace as ss
import ModelBuild as Builder
import TrainModel as Trainer
from google.colab import drive
import pandas as pd
import os
import torch
import random
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms

drive.mount('/content/drive')

Output hidden; open in https://colab.research.google.com to view.

In [2]:
random_params = ss.create_param_combinations(strategy="bayesian")
print("bayesion Search Combinations:", random_params)

bayesion Search Combinations: [{'num_layers': 4, 'units_per_layer': 108, 'activation': 'relu', 'learning_rate': 0.0005829011452567019, 'batch_size': 64, 'dropout_rate': 0.5776722153152961, 'l2_reg_strength': 3.773291459887234e-07}]


In [3]:
# 定义 activation 函数的映射字典
activation_to_num = {'relu': 1, 'tanh': 2, 'sigmoid': 3}
num_to_activation = {1: 'relu', 2: 'tanh', 3: 'sigmoid'}

def initialize_population(num_points, search_space):
    """
    初始化初始种群，生成包含激活函数数字编码的初始点。
    """
    population = [random_sample_point(search_space) for _ in range(num_points)]
    for params in population:
        if 'activation' in params:
            params['activation'] = activation_to_num[params['activation']]
    return population

def random_sample_point(search_space):
    """
    从搜索空间中随机生成一个采样点。
    """
    sample = {}
    for param, config in search_space.items():
        if "values" in config:  # 离散值
            sample[param] = np.random.choice(config["values"])
        elif config["type"] == "continuous":  # 连续值
            if config.get("scale") == "log":
                sample[param] = np.exp(np.random.uniform(np.log(config["min"]), np.log(config["max"])))
            else:
                sample[param] = np.random.uniform(config["min"], config["max"])

            if param == "units_per_layer":
                sample[param] = int(sample[param])
    return sample

def select_next_point(gpr, search_space, X_sample, y_sample, n_candidates=100):
    """
    使用期望改进选择下一个采样点。
    """
    candidates = [random_sample_point(search_space) for _ in range(n_candidates)]
    for candidate in candidates:
        if 'activation' in candidate:
            candidate['activation'] = activation_to_num[candidate['activation']]
    candidates = np.array([list(candidate.values()) for candidate in candidates])

    ei = expected_improvement(candidates, X_sample, gpr, y_sample)
    next_point = candidates[np.argmax(ei)]

    # 将选出的下一点转换为字典格式
    next_point_dict = dict(zip(search_space.keys(), next_point))

    next_point_dict['activation'] = int(next_point_dict['activation'])
    next_point_dict['batch_size'] = int(next_point_dict['batch_size'])
    next_point_dict['units_per_layer'] = int(next_point_dict['units_per_layer'])
    next_point_dict['num_layers'] = int(next_point_dict['num_layers'])

    return next_point_dict

def expected_improvement(X, X_sample, gpr, y_sample, xi=0.01):
    """
    计算候选点的期望改进 (Expected Improvement, EI)。
    """
    from scipy.stats import norm
    mu, sigma = gpr.predict(X, return_std=True)
    mu_sample_opt = np.max(y_sample)

    with np.errstate(divide='warn'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0

    return ei



In [4]:
def bayesian_search(train_set, val_set, epochs=10, save_dir="/content/drive/MyDrive/DL_HPO/BayesianResult"):
    search_space = ss.get_search_space("bayesian")

    initial_points = 10
    total_iterations = 1000

    all_results = []
    best_result = None
    best_val_acc = 0.0
    best_model_wts = None
    total_training_time = 0

    # 确保保存路径存在
    os.makedirs(save_dir, exist_ok=True)
    summary_file_path = f"{save_dir}/bayesian_result.csv"

    # 初始化初始种群并将 activation 转换为数值
    initial_population = initialize_population(initial_points, search_space)
    X_train = np.array([list(point.values()) for point in initial_population])
    y_train = []

    # 代理模型：高斯过程回归
    kernel = Matern(nu=2.5)
    gpr = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, n_restarts_optimizer=10)

    for iteration in range(total_iterations):
        print(f"Iteration {iteration + 1}/{total_iterations}")

        # 初始点或采集函数生成的点
        if iteration < initial_points:
            params = initial_population[iteration]
        else:
            params = select_next_point(gpr, search_space, X_train, y_train)

        print(params)
        params_build = params.copy()
        if 'activation' in params_build:
          params_build['activation'] = num_to_activation[params_build['activation']]


        # 构建模型并加载数据
        model = Builder.build_model(params_build)
        batch_size = int(params["batch_size"])
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

        # 训练模型
        result = Trainer.train_model(model, train_loader, val_loader, epochs=epochs)

        # 累加当前模型的训练时间
        total_training_time += result["training_time"]

        # 记录结果
        result_summary = {
            "iteration": iteration,
            "params": params,
            "strategy": "bayesian",
            "train_loss": result["train_losses"][-1],
            "val_loss": result["val_losses"][-1],
            "train_accuracy": result["train_accuracies"][-1],
            "val_accuracy": result["val_accuracies"][-1],
            "training_time": result["training_time"],
            "total_training_time": total_training_time
        }
        all_results.append(result_summary)

        # 保存结果到文件
        result_df = pd.DataFrame([result_summary])
        if os.path.exists(summary_file_path):
            result_df.to_csv(summary_file_path, mode='a', header=False, index=False)
        else:
            result_df.to_csv(summary_file_path, mode='w', index=False)

        # 更新最佳模型并保存
        if result["val_accuracies"][-1] > best_val_acc:
            best_val_acc = result["val_accuracies"][-1]
            best_result = result_summary
            best_model_wts = model.state_dict()
            best_result_df = pd.DataFrame([best_result])
            best_result_df.to_csv(f"{save_dir}/best_bayesian_result.csv", index=False)
            torch.save(best_model_wts, f"{save_dir}/best_model_weights.pt")

        # 更新代理模型
        print(all_results[0])
        X_train = np.array([list(res['params'].values()) for res in all_results])
        y_train = np.array([res['val_accuracy'] for res in all_results])
        gpr.fit(X_train, y_train)

    return all_results, best_result, best_model_wts


In [5]:
# 下载并预处理 MNIST 数据集
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 加载 MNIST 数据集
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 截取 1000 个训练样本和 100 个验证样本
train_subset = Subset(train_dataset, range(3000))
val_subset = Subset(test_dataset, range(500))

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 36.3MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 1.01MB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 8.39MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 4.16MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
all_results, best_result, best_model_wts = bayesian_search(train_subset, val_subset, epochs=10,save_dir="/content/drive/MyDrive/DL_HPO/BayesianResult2")

Iteration 1/1000
{'num_layers': 3, 'units_per_layer': 15, 'activation': 1, 'learning_rate': 0.11572769949166002, 'batch_size': 64, 'dropout_rate': 0.570402391979106, 'l2_reg_strength': 0.0001459203222690064}
{'iteration': 0, 'params': {'num_layers': 3, 'units_per_layer': 15, 'activation': 1, 'learning_rate': 0.11572769949166002, 'batch_size': 64, 'dropout_rate': 0.570402391979106, 'l2_reg_strength': 0.0001459203222690064}, 'strategy': 'bayesian', 'train_loss': 2.2198531379699706, 'val_loss': 2.1209178562164306, 'train_accuracy': 14.366666666666667, 'val_accuracy': 21.6, 'training_time': 15.026321172714233, 'total_training_time': 15.026321172714233}
Iteration 2/1000
{'num_layers': 1, 'units_per_layer': 79, 'activation': 1, 'learning_rate': 0.00014620708825200177, 'batch_size': 64, 'dropout_rate': 0.35110782263853474, 'l2_reg_strength': 0.0007540784716011744}
{'iteration': 0, 'params': {'num_layers': 3, 'units_per_layer': 15, 'activation': 1, 'learning_rate': 0.11572769949166002, 'batch_

/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


Iteration 133/1000
{'num_layers': 1, 'units_per_layer': 109, 'activation': 3, 'learning_rate': 0.02286237739322568, 'batch_size': 32, 'dropout_rate': 0.4368752805113197, 'l2_reg_strength': 0.0011505270051254993}
{'iteration': 0, 'params': {'num_layers': 3, 'units_per_layer': 15, 'activation': 1, 'learning_rate': 0.11572769949166002, 'batch_size': 64, 'dropout_rate': 0.570402391979106, 'l2_reg_strength': 0.0001459203222690064}, 'strategy': 'bayesian', 'train_loss': 2.2198531379699706, 'val_loss': 2.1209178562164306, 'train_accuracy': 14.366666666666667, 'val_accuracy': 21.6, 'training_time': 15.026321172714233, 'total_training_time': 15.026321172714233}
Iteration 134/1000
{'num_layers': 2, 'units_per_layer': 111, 'activation': 2, 'learning_rate': 0.03447204489581634, 'batch_size': 128, 'dropout_rate': 0.570297417282609, 'l2_reg_strength': 1.6776115304347982e-05}
{'iteration': 0, 'params': {'num_layers': 3, 'units_per_layer': 15, 'activation': 1, 'learning_rate': 0.11572769949166002, 'ba

/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


Iteration 138/1000
{'num_layers': 1, 'units_per_layer': 37, 'activation': 1, 'learning_rate': 0.0003299364463926733, 'batch_size': 128, 'dropout_rate': 0.28730154464674457, 'l2_reg_strength': 2.267211278805207e-05}
{'iteration': 0, 'params': {'num_layers': 3, 'units_per_layer': 15, 'activation': 1, 'learning_rate': 0.11572769949166002, 'batch_size': 64, 'dropout_rate': 0.570402391979106, 'l2_reg_strength': 0.0001459203222690064}, 'strategy': 'bayesian', 'train_loss': 2.2198531379699706, 'val_loss': 2.1209178562164306, 'train_accuracy': 14.366666666666667, 'val_accuracy': 21.6, 'training_time': 15.026321172714233, 'total_training_time': 15.026321172714233}
Iteration 139/1000
{'num_layers': 2, 'units_per_layer': 38, 'activation': 2, 'learning_rate': 0.01299033915223949, 'batch_size': 32, 'dropout_rate': 0.3532406333515858, 'l2_reg_strength': 2.5232445763176185e-07}
{'iteration': 0, 'params': {'num_layers': 3, 'units_per_layer': 15, 'activation': 1, 'learning_rate': 0.11572769949166002, '

...

Iteration 785/1000
{'num_layers': 1, 'units_per_layer': 106, 'activation': 3, 'learning_rate': 0.15775597189739168, 'batch_size': 32, 'dropout_rate': 0.537452957889835, 'l2_reg_strength': 1.5959392221947793e-06}
{'iteration': 0, 'params': {'num_layers': 3, 'units_per_layer': 15, 'activation': 1, 'learning_rate': 0.11572769949166002, 'batch_size': 64, 'dropout_rate': 0.570402391979106, 'l2_reg_strength': 0.0001459203222690064}, 'strategy': 'bayesian', 'train_loss': 2.2198531379699706, 'val_loss': 2.1209178562164306, 'train_accuracy': 14.366666666666667, 'val_accuracy': 21.6, 'training_time': 15.026321172714233, 'total_training_time': 15.026321172714233}


/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


Iteration 786/1000
{'num_layers': 2, 'units_per_layer': 126, 'activation': 2, 'learning_rate': 0.2758538204970659, 'batch_size': 256, 'dropout_rate': 0.4566173626820746, 'l2_reg_strength': 1.8630613053450102e-06}
{'iteration': 0, 'params': {'num_layers': 3, 'units_per_layer': 15, 'activation': 1, 'learning_rate': 0.11572769949166002, 'batch_size': 64, 'dropout_rate': 0.570402391979106, 'l2_reg_strength': 0.0001459203222690064}, 'strategy': 'bayesian', 'train_loss': 2.2198531379699706, 'val_loss': 2.1209178562164306, 'train_accuracy': 14.366666666666667, 'val_accuracy': 21.6, 'training_time': 15.026321172714233, 'total_training_time': 15.026321172714233}
Iteration 787/1000
{'num_layers': 3, 'units_per_layer': 115, 'activation': 1, 'learning_rate': 0.19930527219351823, 'batch_size': 16, 'dropout_rate': 0.10018196316302397, 'l2_reg_strength': 1.5423098714882674e-06}
{'iteration': 0, 'params': {'num_layers': 3, 'units_per_layer': 15, 'activation': 1, 'learning_rate': 0.11572769949166002, '

/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


Iteration 860/1000
{'num_layers': 2, 'units_per_layer': 107, 'activation': 2, 'learning_rate': 0.008735989442473932, 'batch_size': 128, 'dropout_rate': 0.47560058608336614, 'l2_reg_strength': 1.4279634509481655e-05}
{'iteration': 0, 'params': {'num_layers': 3, 'units_per_layer': 15, 'activation': 1, 'learning_rate': 0.11572769949166002, 'batch_size': 64, 'dropout_rate': 0.570402391979106, 'l2_reg_strength': 0.0001459203222690064}, 'strategy': 'bayesian', 'train_loss': 2.2198531379699706, 'val_loss': 2.1209178562164306, 'train_accuracy': 14.366666666666667, 'val_accuracy': 21.6, 'training_time': 15.026321172714233, 'total_training_time': 15.026321172714233}
Iteration 861/1000
{'num_layers': 1, 'units_per_layer': 92, 'activation': 1, 'learning_rate': 0.0319459658962734, 'batch_size': 64, 'dropout_rate': 0.23875507108397692, 'l2_reg_strength': 0.000141064912206128}
{'iteration': 0, 'params': {'num_layers': 3, 'units_per_layer': 15, 'activation': 1, 'learning_rate': 0.11572769949166002, 'b